In [ ]:
# import libraries
from functools import partial
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll.base import scope
import optuna

In [ ]:
# read the files
df = pd.read_csv('../input/mobile-price-classification/train.csv')
X = df.drop('price_range', axis=1).values
y = df['price_range'].values

In [ ]:
# define an optimization function
def optimize(trial, x, y):
    # hyperparameters to optimize
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])
    n_estimators = trial.suggest_int('n_estimators', 100, 1500)
    max_depth = trial.suggest_int('max_depth', 3, 15)
    max_features = trial.suggest_uniform('max_features', 0.01, 1)

    # use Random Forest Classifier
    model = ensemble.RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_features=max_features,
        criterion=criterion,
    )
    
    # use Stratified K-fold
    kf = model_selection.StratifiedKFold(n_splits=5)
    accuracies = []
    for idx in kf.split(X=x, y=y):
        train_idx, val_idx = idx[0], idx[1]
        xtrain = x[train_idx]
        ytrain = y[train_idx]

        xval = x[val_idx]
        yval = y[val_idx]

        model.fit(xtrain, ytrain)
        preds = model.predict(xval) # here we assume to use 0.5 threshold, otherwise you can use predict_proba

        fold_acc = metrics.accuracy_score(yval, preds)
        accuracies.append(fold_acc)

    return -1.0 * np.mean(accuracies) # multiplied by -1 because the function finds the minimum.

In [ ]:
optimization_function = partial(optimize, x=X, y=y)
# you can use maximized, but you have to remove the -1 in the otimization function
study = optuna.create_study(direction='minimize')
study.optimize(optimization_function, n_trials=15)